In [1]:
import random 
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Starter code 

words = open('names.txt', 'r').read().splitlines()

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)

random.seed(42)
random.shuffle(words)

block_size = 8 # Context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # Crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


1. Beat the validation loss of 1.993 by hyperparameter tuning. 

In [3]:
#---------------------------------------------------------------------------------------------------------------------
class Linear:

    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None 
    
    def __call__(self, x):
        self.out = x @ self.weight 
        if self.bias is not None:
            self.out += self.bias 
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

#-----------------------------------------------------------------------------------------------------------------------
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps 
        self.momentum = momentum
        self.training = True 
        # Parameters that are trained with backprop 
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # Not trained with backprop 
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim) 

    def __call__(self, x):
        if self.training:
            if x.ndim == 2:
                dim = 0 
            elif x.ndim == 3:
                dim = (0,1)
            xmean = x.mean(dim, keepdim=True)
            xvar = x.var(dim, keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma*xhat + self.beta 

        if self.training:
            with torch.no_grad():
                self.running_mean = (1-self.momentum)*self.running_mean + self.momentum*xmean 
                self.running_var = (1-self.momentum)*self.running_var + self.momentum*xvar 
    
        return self.out 
        
    def parameters(self):
        return [self.gamma, self.beta]

#-------------------------------------------------------------------------------------------------------------------
class Tanh:

    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out 

    def parameters(self):
        return []

#---------------------------------------------------------------------------------------------------------------------
class Embedding: 

    def __init__(self, num_embeddings, embedding_dimension):
        self.weight = torch.randn(num_embeddings, embedding_dimension)

    def __call__(self, IX):
        self.out = self.weight[IX]
        return self.out 

    def parameters(self):
        return [self.weight]

#-----------------------------------------------------------------------------------------------------------------------
class FlattenConsecutive:

    def __init__(self, n):
        self.n = n 

    def __call__(self, x):
        B, T, C = x.shape
        x = x.view(B, T//self.n, C*self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
        self.out = x 
        return self.out 

    def parameters(self):
        return []

#------------------------------------------------------------------------------------------------------------------------
class Sequential:

    def __init__(self, layers):
        self.layers = layers 
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        self.out = x 
        return self.out 

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [6]:
n_embd = 10
n_hidden = 100

model = Sequential([
    Embedding(vocab_size, n_embd), 
    FlattenConsecutive(2), Linear(n_embd*2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),  
    FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    Linear(n_hidden, vocab_size)
])

with torch.no_grad():
    model.layers[-1].weight *= 0.1 # last layer is less confident 

parameters = model.parameters()
parameters
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

45597


In [7]:
# same optimization as last time
max_steps = 200000
batch_size = 50
lossi = []
ud = []

for i in range(max_steps):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,))
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  # forward pass
  logits = model(Xb)
  loss = F.cross_entropy(logits, Yb)
  
  # backward pass
  for layer in model.layers:
    layer.out.retain_grad() # AFTER_DEBUG: would take out retain_graph
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())
  with torch.no_grad():
    ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])
  

      0/ 200000: 3.2989
  10000/ 200000: 2.0005
  20000/ 200000: 2.1544
  30000/ 200000: 1.7780
  40000/ 200000: 1.9374
  50000/ 200000: 2.1664
  60000/ 200000: 2.1609
  70000/ 200000: 2.0009
  80000/ 200000: 1.8707
  90000/ 200000: 1.9339
 100000/ 200000: 1.7005
 110000/ 200000: 2.2168
 120000/ 200000: 2.1594
 130000/ 200000: 2.0650
 140000/ 200000: 1.7714
 150000/ 200000: 1.9935
 160000/ 200000: 1.8183
 170000/ 200000: 1.4657
 180000/ 200000: 2.0213
 190000/ 200000: 1.4537


In [ ]:
@torch.no_grad()
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr), 
        'val': (Xdev, Ydev), 
        'test': (Xte, Yte)}[split]
    logits = model(x)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 1.7742013931274414
val 2.015970468521118


In [ ]:
# sample from the model
for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass the neural net
      logits = model(torch.tensor([context]))
      probs = F.softmax(logits, dim=1)
      # sample from the distribution
      ix = torch.multinomial(probs, num_samples=1).item()
      # shift the context window and track the samples
      context = context[1:] + [ix]
      out.append(ix)
      # if we sample the special '.' token, break
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out)) # decode and print the generated wor

C:\Users\srira\AppData\Local\Temp\ipykernel_20192\3397384624.py:38: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\ReduceOps.cpp:1823.)
  xvar = x.var(dim, keepdim=True)


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0